In [9]:
import os, sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from skimage import io

# out = widgets.Output()

from lib.labelling import *


In [10]:
# from IPython.display import clear_output
# from matplotlib.widgets import LassoSelector
# from matplotlib.path import Path





# out.clear_output()
plt.close('all')
obj = image_segmenter('new_im', ['mother','daughter'])
# zoom_factory(obj.ax)

from sidecar import Sidecar
sc = Sidecar(title='Segmentation area')
with sc:
    display(obj)
# out

In [61]:
obj.save_mask()
# io.imsave('test-mask.png', obj.class_mask)

In [7]:
io.imsave('masks/5_83490250_z_0.25.png', obj.class_mask)

<ipython-input-7-78a47a9bc16e>:1: UserWarning: masks/5_83490250_z_0.25.png is a low contrast image
  io.imsave('masks/5_83490250_z_0.25.png', obj.class_mask)


https://stackoverflow.com/questions/58050113/imagedatagenerator-for-semantic-segmentation

In [8]:
plt.figure()
plt.imshow(io.imread('masks/5_83490250_z_0.25.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …